In [33]:
import pandas as pd
from newspaper import Article
import nltk
import ast
import requests
from tqdm.auto import tqdm
tqdm.pandas()
from bs4 import BeautifulSoup, Comment
import numpy as np

In [34]:
headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

In [35]:
df = pd.read_csv('KathmanduPost.csv')

In [36]:
# new = pd.read_csv("KathmanduPost_articles.csv")

In [37]:
# joined = df[['id', 'created_at', 'urls']].set_index('id').join(new[['id', 'article']].set_index('id')).reset_index()

In [38]:
# joined[joined.article!='404'].to_csv('KathmanduPost.csv', index=False)

In [40]:
df.shape

(29565, 4)

In [41]:
df.head()

,id,created_at,urls,article
0,1376943554061348864,2021-03-30 22:50:00+05:45,http://tkpo.st/31u5qcO,National A Qatari woman aims to climb Everest ...
1,1376880639417323524,2021-03-30 18:40:00+05:45,https://tkpo.st/3sBwym6,World Myanmar protesters launch 'garbage strik...
2,1376861721130790912,2021-03-30 17:24:49+05:45,https://tkpo.st/3fnEiEw,"Health Nepal’s Covid-19 toll reaches 3,030 wit..."
3,1376807388439699456,2021-03-30 13:48:55+05:45,https://tkpo.st/31xfdPe,National Dahal files petition at Supreme Court...
4,1376800108574957569,2021-03-30 13:20:00+05:45,https://tkpo.st/39qOokl,Sports Nepal edge Bangladesh to claim Three Na...


In [42]:
df.tail()

,id,created_at,urls,article
29560,947657136892346368,2018-01-01 08:19:02+05:45,http://kathmandupost.ekantipur.com/news/2018-0...,NaN
29561,947654359294382080,2018-01-01 08:08:00+05:45,http://kathmandupost.ekantipur.com/news/2018-0...,NaN
29562,947652345613246464,2018-01-01 08:00:00+05:45,http://kathmandupost.ekantipur.com/news/2017-1...,NaN
29563,947637246810775552,2018-01-01 07:00:00+05:45,http://kathmandupost.ekantipur.com/news/2017-1...,NaN
29564,947622146783809537,2018-01-01 06:00:00+05:45,http://kathmandupost.ekantipur.com/news/2017-1...,NaN


## Chunk

In [43]:
def article_ktmpost(row):
    try:
        if (row.article != row.article):
            url = row.urls
            req = requests.get(url, headers)
            soup = BeautifulSoup(req.content, 'html.parser')
            result = soup.find("div", {"class": "col-sm-8"})
            return result.text
        return row.article
    except Exception as e:
        print (str(e))
        return 404

In [44]:
file_path = "KathmanduPost.csv"
result_file_path = "KathmanduPost_articles_continue.csv"

pd.read_csv(file_path, nrows=1).head(0).to_csv(result_file_path)
dataframe = pd.read_csv(file_path, iterator=True, chunksize=2000)

for chunk in dataframe:
   chunk['article'] = chunk.progress_apply(article_ktmpost, axis=1)
   chunk.to_csv(result_file_path, mode='a', header=None)

'NoneType' object has no attribute 'text'



'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text

'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'



HTTPConnectionPool(host='photo.circle', port=80): Max retries exceeded with url: /?Access-Control-Allow-Origin=%2A&Access-Control-Allow-Methods=GET&Access-Control-Allow-Headers=Content-Type&Access-Control-Max-Age=3600&User-Agent=Mozilla%2F5.0+%28X11%3B+Ubuntu%3B+Linux+x86_64%3B+rv%3A52.0%29+Gecko%2F20100101+Firefox%2F52.0 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fee69f283a0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
'NoneType' object has no attribute 'text'
HTTPConnectionPool(host='photo.circle', port=80): Max retries exceeded with url: /?Access-Control-Allow-Origin=%2A&Access-Control-Allow-Methods=GET&Access-Control-Allow-Headers=Content-Type&Access-Control-Max-Age=3600&User-Agent=Mozilla%2F5.0+%28X11%3B+Ubuntu%3B+Linux+x86_64%3B+rv%3A52.0%29+Gecko%2F20100101+Firefox%2F52.0 (Caused by NewConnectionError('<urllib3.con

'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'



In [46]:
new = pd.read_csv(result_file_path)

In [50]:
new.shape

(29565, 5)

In [52]:
new[new.article!='404'][['id', 'created_at', 'urls', 'article']].to_csv('KathmanduPost.csv', index=False)